In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from sklearn.svm import SVC
import json

In [2]:
features_angles = pickle.load(open("features_angles.pkl", "rb"))
test_fv = pickle.load(open("test_fv.pkl", "rb"))
train_labels = pickle.load(open("train_labels.pkl", "rb"))

In [3]:
len(features_angles)

92

In [11]:
len(features_angles[0])

870

In [60]:
train_labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 7., 7., 7.,
       7., 7., 7., 7., 7., 7., 7., 8., 8., 8., 8., 8., 8., 8., 8., 8., 9.,
       9., 9., 9., 9., 9., 9., 9.])

In [5]:
test_fv = [vector.reshape(10,29,3) for vector in test_fv]

In [7]:
test_fv = np.array(test_fv)

In [8]:
test_fv.shape

(1, 10, 29, 3)

In [16]:
new_features_angles = [vector.reshape(10,29,3) for vector in features_angles]

In [17]:
new_features_angles = np.array(new_features_angles)

In [14]:
clf = SVC(kernel="linear")

In [15]:
clf.fit(features_angles,train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [21]:
clf.predict(test_fv)[0]

9.0

In [30]:
a = features_angles[0]

In [34]:
a

array([-0.03148783,  0.05435647, -0.03103553, -0.01694534, -0.02455837,
       -0.0289781 , -0.02984959, -0.02570659, -0.0185977 , -0.02783579,
       -0.03315584, -0.03608555, -0.03319766, -0.02584614,  0.07335605,
       -0.03576313, -0.02920351,  0.0480683 , -0.0185548 , -0.02448564,
       -0.00727916,  0.08347139,  0.02965541, -0.03041024, -0.03216949,
        0.01411849, -0.03568234,  0.06043157,  0.02990102,  0.03728727,
        0.00153906, -0.00397001,  0.00338048, -0.01061022, -0.01080408,
       -0.01080408, -0.00514726, -0.0082804 ,  0.0041452 , -0.01220798,
       -0.00639831,  0.00639671,  0.00216365,  0.00962088,  0.        ,
       -0.01209616,  0.0197729 ,  0.0328069 ,  0.02736929,  0.02736929,
        0.060836  ,  0.        ,  0.05783522,  0.03793814, -0.0384714 ,
        0.06451139, -0.01940754,  0.05368733, -0.00167791,  0.00757988,
       -0.00680986,  0.00680989, -0.00732926, -0.00732926, -0.0019771 ,
        0.00529013, -0.00672785,  0.00945391, -0.00964202, -0.01

In [38]:
b = a.reshape(10,29,3)

In [40]:
b.shape

(10, 29, 3)

In [4]:
num_classes=10
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
image_size = 72  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [5]:
num_patches

144

In [6]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [7]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [8]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [9]:
data_augmentation = keras.Sequential(
    [
#         layers.Normalization(),
#         layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(features_angles)

In [19]:
data_augmentation

In [15]:
def create_vit_classifier():
    inputs = layers.Input(shape=(870,))
    # Augment data.
#     augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [16]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "./tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=new_features_angles,
        y=train_labels,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history

In [17]:
vit_classifier = create_vit_classifier()

ValueError: in user code:

    <ipython-input-7-68cea0bafeb2>:8 call  *
        patches = tf.image.extract_patches(
    C:\Users\bruno belluzzo\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\util\dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    C:\Users\bruno belluzzo\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\array_ops.py:6249 extract_image_patches_v2
        padding, name)
    C:\Users\bruno belluzzo\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\gen_array_ops.py:2388 extract_image_patches
        rates=rates, padding=padding, name=name)
    C:\Users\bruno belluzzo\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\op_def_library.py:750 _apply_op_helper
        attrs=attr_protos, op_def=op_def)
    C:\Users\bruno belluzzo\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\func_graph.py:601 _create_op_internal
        compute_device)
    C:\Users\bruno bel

In [83]:
history = run_experiment(vit_classifier)

Epoch 1/100
1/1 [==============================] - 13s 13s/step - loss: 5.0838 - accuracy: 0.0732 - top-5-accuracy: 0.4512 - val_loss: 14.0781 - val_accuracy: 0.0000e+00 - val_top-5-accuracy: 0.2000
Epoch 2/100
1/1 [==============================] - 3s 3s/step - loss: 3.8542 - accuracy: 0.1585 - top-5-accuracy: 0.7317 - val_loss: 14.6006 - val_accuracy: 0.0000e+00 - val_top-5-accuracy: 0.2000
Epoch 3/100
1/1 [==============================] - 3s 3s/step - loss: 3.5678 - accuracy: 0.2317 - top-5-accuracy: 0.7195 - val_loss: 15.7181 - val_accuracy: 0.0000e+00 - val_top-5-accuracy: 0.2000
Epoch 4/100
1/1 [==============================] - 3s 3s/step - loss: 3.2761 - accuracy: 0.2073 - top-5-accuracy: 0.8293 - val_loss: 16.5715 - val_accuracy: 0.0000e+00 - val_top-5-accuracy: 0.2000
Epoch 5/100
1/1 [==============================] - 3s 3s/step - loss: 2.7188 - accuracy: 0.3171 - top-5-accuracy: 0.8171 - val_loss: 14.2665 - val_accuracy: 0.1000 - val_top-5-accuracy: 0.2000
Epoch 6/100
1/1 [

1/1 [==============================] - 3s 3s/step - loss: 0.0764 - accuracy: 0.9756 - top-5-accuracy: 1.0000 - val_loss: 24.6348 - val_accuracy: 0.2000 - val_top-5-accuracy: 0.2000
Epoch 86/100
1/1 [==============================] - 3s 3s/step - loss: 0.1674 - accuracy: 0.9634 - top-5-accuracy: 1.0000 - val_loss: 25.1139 - val_accuracy: 0.2000 - val_top-5-accuracy: 0.2000
Epoch 87/100
1/1 [==============================] - 3s 3s/step - loss: 0.3740 - accuracy: 0.9146 - top-5-accuracy: 1.0000 - val_loss: 23.9773 - val_accuracy: 0.2000 - val_top-5-accuracy: 0.2000
Epoch 88/100
1/1 [==============================] - 3s 3s/step - loss: 0.2120 - accuracy: 0.9512 - top-5-accuracy: 1.0000 - val_loss: 22.4377 - val_accuracy: 0.2000 - val_top-5-accuracy: 0.2000
Epoch 89/100
1/1 [==============================] - 3s 3s/step - loss: 0.3069 - accuracy: 0.9390 - top-5-accuracy: 1.0000 - val_loss: 21.1069 - val_accuracy: 0.2000 - val_top-5-accuracy: 0.2000
Epoch 90/100
1/1 [=========================

NameError: name 'x_test' is not defined

In [22]:
vit_classifier.load_weights("./tmp/checkpoint")

In [86]:
model.evaluate(new_features_angles)

NameError: name 'model' is not defined

In [23]:
resposta = vit_classifier.predict(test_fv)

In [25]:
resposta[0]

array([  3.3984425 ,   7.184504  ,  -0.04978048,   5.6326504 ,
        -3.816053  ,   7.013057  ,  -3.1799543 ,  -5.814621  ,
         8.616629  , -18.301085  ], dtype=float32)

In [27]:
int(resposta[0].argmax())

8

# Text Classification Model

In [22]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [19]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [33]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=2000)

17473536/17464789 [==============================] - 2s 0us/step


In [36]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=200)

In [105]:
x_train

array([[  5,  25, 100, ...,  19, 178,  32],
       [  0,   0,   0, ...,  16, 145,  95],
       [  0,   0,   0, ...,   7, 129, 113],
       ...,
       [  0,   0,   0, ...,   4,   2,   2],
       [  0,   0,   0, ...,  12,   9,  23],
       [  0,   0,   0, ..., 204, 131,   9]])

In [40]:
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=200)

In [47]:
x_train.shape

(25000, 200)

In [48]:
features_angles.shape

(92, 870)

In [115]:
embed_dim = 96  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(870,))
embedding_layer = TokenAndPositionEmbedding(870, 2000, 96)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [116]:
x_train[0]

array([   5,   25,  100,   43,  838,  112,   50,  670,    2,    9,   35,
        480,  284,    5,  150,    4,  172,  112,  167,    2,  336,  385,
         39,    4,  172,    2, 1111,   17,  546,   38,   13,  447,    4,
        192,   50,   16,    6,  147,    2,   19,   14,   22,    4, 1920,
          2,  469,    4,   22,   71,   87,   12,   16,   43,  530,   38,
         76,   15,   13, 1247,    4,   22,   17,  515,   17,   12,   16,
        626,   18,    2,    5,   62,  386,   12,    8,  316,    8,  106,
          5,    4,    2,    2,   16,  480,   66,    2,   33,    4,  130,
         12,   16,   38,  619,    5,   25,  124,   51,   36,  135,   48,
         25, 1415,   33,    6,   22,   12,  215,   28,   77,   52,    5,
         14,  407,   16,   82,    2,    8,    4,  107,  117,    2,   15,
        256,    4,    2,    7,    2,    5,  723,   36,   71,   43,  530,
        476,   26,  400,  317,   46,    7,    4,    2, 1029,   13,  104,
         88,    4,  381,   15,  297,   98,   32,   

In [117]:
features_angles[0]

array([-0.03257373,  0.01453273, -0.03061472, -0.02082982,  0.11790981,
       -0.02703652,  0.03362912,  0.03371414, -0.03006495, -0.03442991,
        0.0408205 , -0.03565749,  0.03599875,  0.0981624 ,  0.07726734,
       -0.0374425 , -0.0321108 , -0.03768962, -0.02173856, -0.03717138,
       -0.03183392, -0.02701145, -0.03339936, -0.03402802, -0.02630187,
       -0.02221895, -0.03177619,  0.00994954, -0.03073265, -0.02716654,
       -0.02109196,  0.02010781,  0.01887883, -0.00706891, -0.02915106,
       -0.02915106,  0.02212439, -0.02313733, -0.01751082, -0.02406493,
        0.01694044,  0.00758309, -0.02348271,  0.00658279,  0.01951865,
        0.04501304,  0.06326226,  0.01805053, -0.02921667, -0.02921667,
       -0.00914365, -0.02466349,  0.03104447, -0.0274369 , -0.03935496,
        0.01722397,  0.06138108,  0.02177665,  0.00828035, -0.01002643,
        0.        , -0.00667753,  0.01079002,  0.01079002,  0.00803378,
       -0.00783358,  0.        , -0.00697727,  0.        , -0.00

In [118]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    features_angles, train_labels, batch_size=32, epochs=10
)

Epoch 1/10


InvalidArgumentError:  indices[545] = 545 is not in [0, 200)
	 [[node model_5/token_and_position_embedding_7/embedding_15/embedding_lookup (defined at <ipython-input-22-7321ccbd6d67>:10) ]] [Op:__inference_train_function_28381]

Errors may have originated from an input operation.
Input Source operations connected to node model_5/token_and_position_embedding_7/embedding_15/embedding_lookup:
 model_5/token_and_position_embedding_7/embedding_15/embedding_lookup/27592 (defined at C:\Anaconda\envs\harbop\lib\contextlib.py:112)	
 model_5/token_and_position_embedding_7/range (defined at <ipython-input-22-7321ccbd6d67>:9)

Function call stack:
train_function


In [91]:
train_labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
       3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 7., 7., 7., 7.,
       7., 7., 7., 7., 7., 7., 8., 8., 8., 8., 8., 8., 8., 8., 8., 9., 9.,
       9., 9., 9., 9., 9., 9., 9.])

In [98]:
np.array(features_angles[0]).shape

(870,)

In [110]:
pred = model.predict(np.array(features_angles[0:2]))

In [111]:
len(pred)

2

In [113]:
pred[0]

array([0.09963427, 0.10062379, 0.10103338, 0.09928057, 0.10065974,
       0.09960502, 0.09900425, 0.09938139, 0.10063899, 0.10013861],
      dtype=float32)